In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('level2_i_20221201.csv')
df.head()

a = df['midp'].diff(1200) / df['midp'].shift(1200)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['5min return'] = a

b = df['midp'].diff(240) / df['midp'].shift(240)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['1min return'] = b

In [3]:
col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('5min return')
col_reserved.append('1min return')


In [4]:
new_df = df[col_reserved]
linear_df = new_df[1199:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)
linear_df

,fac1,fac2,fac3,fac4,fac5,fac6,fac7,fac8,fac9,fac10,...,fac26,fac27,fac28,fac29,fac30,fac31,fac32,fac33,5min return,1min return
0,-0.000201,-0.100503,-1.000000,-0.039539,-0.008035,0.012771,0.017750,0.005622,-0.005422,4.0,...,0.0,0.000000,-0.002507,0.000537,0.000537,0.000495,0.000230,-0.000046,0.000647,-0.000647
1,-0.000214,-0.019506,-0.555555,-0.044372,-0.006845,0.012014,0.017698,0.005543,-0.005437,3.0,...,0.0,-0.028571,-0.002507,0.000537,0.000537,0.000493,0.000235,0.000121,0.000647,-0.000647
2,-0.000292,-0.132730,-0.830189,-0.068396,-0.011363,0.008915,0.016692,0.004849,-0.005456,2.0,...,0.0,-0.048544,-0.002496,0.000523,0.000523,0.000471,0.000226,0.000183,0.000647,-0.000647
3,-0.000296,-0.017991,-0.999999,-0.067358,-0.010600,0.007809,0.016580,0.004801,-0.005237,0.0,...,0.0,-0.039216,-0.002493,0.000523,0.000523,0.000468,0.000250,0.000201,0.000647,-0.000647
4,-0.000290,-0.021552,-0.517241,-0.046512,-0.010224,0.007456,0.016282,0.004631,-0.005729,0.0,...,0.0,-0.049505,-0.002485,0.000524,0.000524,0.000468,0.000250,0.000234,0.000647,-0.000647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74346,-0.000054,0.001085,0.333332,0.000816,-0.002180,0.000699,0.004050,-0.004290,-0.006255,0.0,...,-0.0,-0.333333,-0.000835,0.000040,0.000040,0.000171,0.000224,0.000326,-0.000653,0.000653
74347,-0.000054,-0.001086,-0.999990,0.000815,-0.002074,0.000458,0.004056,-0.004131,-0.006255,0.0,...,-0.0,-0.333333,-0.000835,0.000044,0.000044,0.000171,0.000223,0.000326,-0.000653,0.000653
74348,-0.000054,-0.001085,-0.999990,0.000271,-0.001803,0.000459,0.004125,-0.003815,-0.006258,0.0,...,-0.0,-0.333333,-0.000835,0.000053,0.000053,0.000171,0.000223,0.000326,-0.000653,0.000653
74349,-0.000053,0.004329,0.999998,0.000813,0.000382,0.007821,0.004176,-0.002501,-0.006255,1.0,...,0.0,-0.333333,-0.000835,0.000134,0.000134,0.000169,0.000036,0.000109,-0.000653,0.000653


In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

window_size = 200

predict_5min = []
predict_1min = []
# this sequence record the R square of validation set
R_square_5min = []
R_square_1min = []

for i in range(window_size, len(linear_df)):
    train_X = linear_df.iloc[i - window_size : i, :-2]
    train_y_5min = linear_df.iloc[i - window_size : i, -2]
    train_y_1min = linear_df.iloc[i - window_size : i, -1]

    train_X_5min, validate_X_5min, train_y_5min, validate_y_5min = train_test_split(train_X, train_y_5min, test_size = 0.1, random_state = 11)
    train_X_1min, validate_X_1min, train_y_1min, validate_y_1min = train_test_split(train_X, train_y_1min, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_5min = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_1min = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    model_1min = LinearRegression()
    model_5min = LinearRegression()
    
    scaler_1min = StandardScaler()
    scaler_5min = StandardScaler()

    scaled_train_X_1min = scaler_1min.fit_transform(train_X_1min)
    scaled_validate_X_1min = scaler_1min.transform(validate_X_1min)
    scaled_test_X_1min = scaler_1min.transform(test_X)

    scaled_train_X_5min = scaler_5min.fit_transform(train_X_5min)
    scaled_validate_X_5min = scaler_5min.transform(validate_X_5min)
    scaled_test_X_5min = scaler_5min.transform(test_X)

    model_1min.fit(scaled_train_X_1min, train_y_1min)
    model_5min.fit(scaled_train_X_5min, train_y_5min)

    predict_1min.append(model_1min.predict(scaled_test_X_1min))
    predict_5min.append(model_5min.predict(scaled_test_X_5min))

    R_square_1min.append(model_1min.score(scaled_validate_X_1min, validate_y_1min))
    R_square_5min.append(model_5min.score(scaled_validate_X_5min, validate_y_5min))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_1min = {}'.format(model_1min.score(scaled_validate_X_1min, validate_y_1min)))
        print('R_square_5min = {}'.format(model_5min.score(scaled_validate_X_5min, validate_y_5min)))


i = 500
R_square_1min = 0.9532249894989724
R_square_5min = 0.8408826800793119
i = 1000
R_square_1min = 0.9518629086924353
R_square_5min = 0.9105138082521089
i = 1500
R_square_1min = 0.9330330016989541
R_square_5min = 0.9367596271194345
i = 2000
R_square_1min = 0.7757595342494902
R_square_5min = 0.9496619420544116
i = 2500
R_square_1min = 0.6153008610957782
R_square_5min = 0.954981709344165
i = 3000
R_square_1min = 0.9106661271266704
R_square_5min = 0.8105125668863966
i = 3500
R_square_1min = 0.9884930635708553
R_square_5min = 0.9876735886117304
i = 4000
R_square_1min = 0.6635111301788224
R_square_5min = 0.4676524073438342
i = 4500
R_square_1min = 0.6418231447263401
R_square_5min = 0.8191002795349095
i = 5000
R_square_1min = 0.7943841473284363
R_square_5min = 0.8687550959319117
i = 5500
R_square_1min = 0.8586196502255535
R_square_5min = 0.8330569043577383
i = 6000
R_square_1min = 0.8068598450162301
R_square_5min = 0.8337020693455197
i = 6500
R_square_1min = 0.9967164244303275
R_square_5

In [49]:
predict_1 = np.array(predict_1min).reshape(-1)
predict_5 = np.array(predict_5min).reshape(-1)
R_square_1 = np.array(R_square_1min).reshape(-1)
R_square_5 = np.array(R_square_5min).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_1min'] = predict_1
new_df['predict_5min'] = predict_5
new_df['R_square_1min'] = R_square_1
new_df['R_square_5min'] = R_square_5



In [50]:
new_df

,predict_1min,predict_5min,R_square_1min,R_square_5min
0,-0.000627,0.001361,0.611734,0.822458
1,-0.000377,0.001391,0.604159,0.825487
2,-0.000206,0.001449,0.613147,0.861153
3,-0.000196,0.001424,0.574436,0.868442
4,-0.000146,0.001431,0.569849,0.902526
...,...,...,...,...
9795,0.000696,-0.001339,0.955667,0.978052
9796,0.000604,-0.001336,0.936524,0.956262
9797,0.000708,-0.001329,0.937681,0.896116
9798,0.000634,-0.001010,0.938435,0.984893
